In [15]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [16]:
from data_manager.database_manager import db_fetchone,db_no_fetch
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        display('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor() as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchall()[0]
                display(f"PostgreSQL database version: {db_version}")
                display(f"PostgreSQL connection status: {conn.info.status}")
                display(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        display(error)
        conn = None


In [17]:
conn=init_connection(dsn)

'Connecting to the PostgreSQL database...'

"PostgreSQL database version: ('PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)"

'PostgreSQL connection status: 0'

"You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'."

In [30]:
def db_fetchone(sql_message, conn,vars: List=None):
    with conn:
        with conn.cursor() as cur:
            try:
                if vars:
                    cur.execute(sql_message, vars)
                else:
                    cur.execute(sql_message)
                conn.commit()
                return_one = cur.fetchone()  # return tuple
                return return_one
            except psycopg2.Error as e:
                display(e)

In [37]:
def project_dataset(dataset_chosen,conn,project_id):
    insert_project_dataset_SQL = """
                                        INSERT INTO public.project_dataset (
                                            project_id,
                                            dataset_id)
                                        VALUES (
                                            %s,
                                            %s);"""
    for dataset in dataset_chosen:
        dataset_id = dataset_name_id[dataset]
        insert_project_dataset_vars = [project_id, dataset_id]
        db_fetchone(insert_project_dataset_SQL, conn,
                        insert_project_dataset_vars)

In [32]:
dataset_chosen=['My Second Dataset','My Third Dataset']
dataset_name_id={'My Second Dataset':2,'My Third Dataset':3,'My Fourth Dataset':4}

In [33]:
project_id=3
for dataset in dataset_chosen:
        dataset_id = dataset_name_id[dataset]
        insert_project_dataset_vars = [project_id, dataset_id]
        print(insert_project_dataset_vars)

[3, 2]
[3, 3]


In [38]:
project_dataset(dataset_chosen,conn,project_id)

psycopg2.ProgrammingError('no results to fetch')

psycopg2.ProgrammingError('no results to fetch')

In [42]:
def init_editor(name,editor_config,project_id,conn) -> int:
        init_editor_SQL = """
                                    INSERT INTO public.editor (
                                        name,
                                        editor_config,
                                        project_id)
                                    VALUES (
                                        %s,
                                        %s,
                                        %s)
                                    RETURNING
                                        id;"""

        init_editor_vars = [name, editor_config,project_id]
        id = db_fetchone(init_editor_SQL,conn,init_editor_vars)
        return id

In [45]:
editor_config="""
<View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""
project_id=5

In [46]:
init_editor('sfsfg',editor_config,project_id,conn)[0]

(3,)

# Editor Config

In [32]:
from xml.dom import minidom
import xml.dom

In [49]:
data="""
<View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""

In [50]:
doc=minidom.parseString(data)
doc

In [51]:
doc.getElementsByTagName('RectangleLabels')[0].getElementsByTagName('Label')

[<DOM Element: RectangleLabels at 0x7f81aa4ebe50>,
 <DOM Element: RectangleLabels at 0x7f81aa484ca0>]

In [90]:
parents=doc.getElementsByTagName('RectangleLabels')
elements=[]
for parent in parents:
    childs=parent.getElementsByTagName('Label')
    for child in childs:
        print(child)
        elements.append(child)

elements
tagName_attributes=[]
for element in elements:
    print(element.tagName,element.attributes.items())
    tagName_attributes.append((element.tagName,element.attributes.items()))
    if element.hasAttribute('value'):
        print((element.getAttribute('value'))) #gives string of value
tagName_attributes


<DOM Element: Label at 0x7f81aa4ebee0>
<DOM Element: Label at 0x7f81a9f7e280>
<DOM Element: Label at 0x7f81aa484430>
<DOM Element: Label at 0x7f81aa484550>
Label [('value', 'Hello'), ('background', 'blue')]
Hello
Label [('value', 'World'), ('background', 'pink')]
World
Label [('value', 'Hello'), ('background', 'blue')]
Hello
Label [('value', 'World'), ('background', 'pink')]
World


[('Label', [('value', 'Hello'), ('background', 'blue')]),
 ('Label', [('value', 'World'), ('background', 'pink')]),
 ('Label', [('value', 'Hello'), ('background', 'blue')]),
 ('Label', [('value', 'World'), ('background', 'pink')])]

In [37]:
doc.childNodes

[<DOM Element: View at 0x7f81aa2f98b0>]

## Create Node

In [11]:
def create_element(doc,parent,tagname,attr,value):
    nodeList = doc.getElementsByTagName(parent)[0]
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    new_label=doc.createElement(tagname)
    new_label.setAttribute(attr,value)
    
    nodeList.appendChild(new_label)

In [12]:
create_element(doc,'RectangleLabels','Label','value','Hello')

In [13]:
rectangle=doc.getElementsByTagName('RectangleLabels')[0]
print(rectangle)
new_label=doc.createElement('Label')
new_label.setAttribute("value","aruco")
new_label.setAttribute('background','red')
rectangle.appendChild(new_label)

<DOM Element: RectangleLabels at 0x7f81aadd1dc0>


<DOM Element: Label at 0x7f81aa692b80>

## Remove Nodes

In [15]:
def removeNodes(doc,tagname,attr,value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            parent = node.parentNode
            parent.removeChild(node)

## Edit Nodes

In [16]:
def set_attribute(doc,tagname,attr,value,new_value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            node.setAttribute(attr,new_value)

In [17]:
set_attribute(doc,'Label','value','Hello','aruco')

In [ ]:
value={'old':'Hello','new':'aruco'}
attrib_value=[value]

In [ ]:
def set_attribute(doc,tagname,attrib_value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            node.setAttribute(attr,new_value)

In [18]:
removeNodes(doc,'Label','value','Hello')

In [139]:
from copy import deepcopy
a=[1,2,3,4]
b=deepcopy(a)
print(b)

b


[1, 2, 3, 4]


[1, 2, 3, 4]

## Print DOM

In [87]:
def xml_string(xml_doc):
    xml_string=xml_doc.toprettyxml(indent="\t",newl=" ",encoding='utf8').decode('utf-8')
    return xml_string

In [140]:
print(xml_string(doc))

<?xml version="1.0" encoding="utf8"?> <View> 	
 	<Header value="Select label and start to click on image"/> 	
   	<View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse"> 		
     		<Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/> 		
     		<View> 			
       			<Filter toName="tag" minlength="0" name="filter"/> 			
       			<RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5"> 				
         				<Label value="Hello" background="blue"/> 				
         				<Label value="World" background="pink"/> 				
       			</RectangleLabels> 			
     		</View> 		
     		<View> 			
       			<Filter toName="tag" minlength="0" name="filter"/> 			
       			<RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5"> 				
         				<Label value="Hello" background="blue"/> 				
         				<Label value="Wo

In [109]:
list1=[1,2,3,6,7,8,9,10]
list2=[1,2,3,6,7,8,9]
diff_12=set(list1).difference(list2) #set 1 - set 2
diff_21=set(list2).difference(list1) #set 2 - set 1
if diff_12:
    print("Removal")
elif diff_21:
    print("Addition")
else:
    print("No Change")

Removal


In [104]:
from core.utils.helper import is_empty
x=[1]
is_empty(x)

False

In [136]:
listA=[1,2,3,6,7,8,9,10]
listB=[1,2,3,6,7,8,9]
listA.remove(10)
# listA=listA+[10]
print(listA)

[1, 2, 3, 6, 7, 8, 9]


In [118]:
list(diff_12).pop()

10

In [13]:
d=['hi','Hi','bye','bye','love']
setted=set(d)
print(setted)
sorted(list(setted))

{'hi', 'bye', 'love', 'Hi'}


['Hi', 'bye', 'hi', 'love']

In [14]:
x='HELLO'
print(x.lower())
x

hello


'HELLO'